This notebook does two things: 1) tracks the co-firing of cells as it increases/decreases on each day, and 2) tests whether day 4 (learning-related) increases in syncrony persist with the same cell pairs between controls and ANI mice.

## NRK todo: remake all analyses not tracking cells across days with d1_d2 as same day, e.g. '4_4'.  The clarity will make life easier in future!

#2 - Figure out plots to add to final figure, layout with gridspec (put stats off to side), and plot with appropriate font size, lineweights, etc. so that you don't have to mess with them at all in illustrator afterward.

#3 - Same thing as #2 but for supplemental figures, e.g. tracking cells across days for control days (e.g. day -1 freeze cells, day -1 cells with high covariance).

In [ ]:
# Import useful modules
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.gridspec as gridspec
import pandas as pd
from tqdm import tqdm

# Import project specific modules and enable automatic reloading
import sys
from pathlib import Path
import os
eraser_path = Path(os.getcwd()).parent
reinstatement_path = eraser_path.parent / 'FearReinstatement'
sys.path.append(str(eraser_path))
sys.path.append(str(reinstatement_path))
%load_ext autoreload
%autoreload 2

from session_directory import find_eraser_directory as get_dir
import er_plot_functions as erp
import Placefields as pf
import freezing_analysis as fa
import helpers 
import eraser_reference as err
import covariance_analysis as cova

# Project save directory here
print_dir = '/Users/nkinsky/Documents/BU/Working/Eraser/plots'
save_dir = '/Users/nkinsky/Documents/BU/Working/Eraser'

plt.rcParams['pdf.fonttype'] = 42
# plt.rcParams['']
plt.rcParams['font.sans-serif'] = "Arial"

In [ ]:
# Helper functions to block printing output
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__


## Bug here - why isn't this fix working!!!
Should be fixed now - re-test all code below with updates, make sure to change 'before', 'during', and 'after' inputs and get rid of any refs to 'add_reg_memory_designation' function

In [ ]:
from copy import deepcopy

def add_memory_designations(df_use, before=['-2_-1', '-1_4'], during=['4_1'], after=['1_2', '2_7']):
    """Add in designations for before, 4hr (during), and after ANI administration into your dataframe"""
    df = deepcopy(df_use)
    desig = ["" for a in df.itertuples()]
    
    def create_bool_array_from_list(bool_list):
        bool_out = False
        for b in bool_list:
            bool_out = np.bitwise_or(bool_out, b)
            
        return bool_out
    
    before_bool_list = [df["d1_d2"] == bef for bef in before]
    before_ind = np.where(create_bool_array_from_list(before_bool_list))[0]
    during_bool_list = [df["d1_d2"] == dur for dur in during]
    during_ind = np.where(create_bool_array_from_list(during_bool_list))[0]
    after_bool_list = [df["d1_d2"] == aft for aft in after]
    after_ind = np.where(create_bool_array_from_list(after_bool_list))[0]
    
       # Old hardcoded method below for reference/bug checking
#     before_ind = np.where((df["d1_d2"] == "-2_-1") | (df["d1_d2"] == "-1_4"))[0]
#     during_ind = np.where(df["d1_d2"] == "4_1")[0]
#     after_ind = np.where((df["d1_d2"] == "1_2") | (df["d1_d2"] == "2_7"))[0]
    for idd, d in enumerate(desig):
        if idd in before_ind:
            desig[idd] = "Before"
        elif idd in during_ind:
            desig[idd] = "4hr"
        elif idd in after_ind:
            desig[idd] = "After"
    df.insert(1, "Designation", desig)
    
    return df

In [ ]:
def stats_before_after(df_use, key_use, ax=None, fontsize=None):
    """Compares mean covariances in a DataFrame. Pools days before (-2 and -1) and days after (1 and 2) shock + 
    injection. Compares across and within groups.
    :param reg: True = use reg_memory_designations, False = use regular memory degisnations"""
    
    df_before = df_use[df_use['Designation'] == 'Before']
    df_after = df_use[df_use['Designation'] == 'After']
    df4 = df_use[df_use['Designation'] == '4hr'] 
        
    stat_bef, p_bef = stats.ttest_ind(df_before[df_before['Group'] == 'Ctrl'][key_use], 
                                     df_before[df_before['Group'] == 'ANI'][key_use], nan_policy='omit')
    stat_aft, p_aft = stats.ttest_ind(df_after[df_after['Group'] == 'Ctrl'][key_use], 
                                     df_after[df_after['Group'] == 'ANI'][key_use], nan_policy='omit')
    stat_ctrl, p_ctrl = stats.ttest_ind(df_before[df_before['Group'] == 'Ctrl'][key_use], 
                                     df_after[df_after['Group'] == 'Ctrl'][key_use], nan_policy='omit')
    stat_ani, p_ani = stats.ttest_ind(df_before[df_before['Group'] == 'ANI'][key_use], 
                                     df_after[df_after['Group'] == 'ANI'][key_use], nan_policy='omit')
    stat_dur, p_dur = stats.ttest_ind(df4[df4['Group'] == 'Ctrl'][key_use],
                                     df4[df4['Group'] == 'ANI'][key_use], nan_policy='omit')
    stat_b4c, p_b4c = stats.ttest_ind(df_before[df_before['Group'] == 'Ctrl'][key_use],
                                      df4[df4['Group'] == 'Ctrl'][key_use], nan_policy='omit')
    stat_b4a, p_b4a = stats.ttest_ind(df_before[df_before['Group'] == 'ANI'][key_use],
                                      df4[df4['Group'] == 'ANI'][key_use], nan_policy='omit')
    stat_4ac, p_4ac = stats.ttest_ind(df4[df4['Group'] == 'Ctrl'][key_use], 
                                     df_after[df_after['Group'] == 'Ctrl'][key_use], nan_policy='omit')
    stat_4aa, p_4aa = stats.ttest_ind(df4[df4['Group'] == 'ANI'][key_use], 
                                     df_after[df_after['Group'] == 'ANI'][key_use], nan_policy='omit')
    
    if ax is None:  # Print to screen
        print(f'p={p_bef:0.3g} baseline b/w groups')
        print(f'p={p_aft:0.3g} day 1-2 b/w groups')
        print(f'p={p_ctrl:0.3g} before-after Ctrl')
        print(f'p={p_ani:0.3g} before-after ANI')
        print(f'p={p_dur:0.3g} 4 hrs b/w groups')
    else:  # Print into specified axes
        if fontsize is None:
            fsize = 12.0
        else:
            fsize = 4.0
        ax.text(0.1, 0.9, f'p={p_4aa:0.3g} 4 hr v after ANI', fontsize=fsize)
        ax.text(0.1, 0.8, f'p={p_4ac:0.3g} 4 hr v after Ctrl', fontsize=fsize)
        ax.text(0.1, 0.7, f'p={p_b4a:0.3g} before v 4 hr ANI', fontsize=fsize)
        ax.text(0.1, 0.6, f'p={p_b4c:0.3g} before v 4 hr Ctrl', fontsize=fsize)
        ax.text(0.1, 0.5, f'p={p_bef:0.3g} baseline b/w groups', fontsize=fsize)
        ax.text(0.1, 0.4, f'p={p_aft:0.3g} day 1-2 b/w groups', fontsize=fsize)
        ax.text(0.1, 0.3, f'p={p_ctrl:0.3g} before-after Ctrl', fontsize=fsize)
        ax.text(0.1, 0.2, f'p={p_ani:0.3g} before-after ANI', fontsize=fsize)
        ax.text(0.1, 0.1, f'p={p_dur:0.3g} 4 hrs b/w groups', fontsize=fsize)

Step 0: Set up all dicts for plotting.

In [ ]:
bin_size = 0.5 # sec
all_cov = cova.group_cov_across_days(bin_size=bin_size, arena1='Shock', arena2='Shock',
                                               neurons='all', keep_silent=True)
freeze_silent_cov = cova.group_cov_across_days(bin_size=bin_size, arena1='Shock', arena2='Shock',
                                               neurons='freeze_onset', keep_silent=True)

Plot #1: Freeze cell covariance before (day -1 and -2), 4 hr, and after (day 1 and 2), all normalized to day -2.  

NRK important note - try with keep_silent=False - all the above is probably driven by silent cells!  They are listed as 0 when they should be NaN by definition since the variance of a cell that fires no events is NaN, right?

### NO! I'm only looking at base day covariances, which don't include silent cells! All good!

In [ ]:
sigz_freeze_df = cova.cov_dict_to_df(freeze_silent_cov, baseline_dict_use=all_cov, include_silent=True)
sigz_freeze_df = add_memory_designations(sigz_freeze_df)

In [ ]:
sigz_df = sigz_freeze_df

pairs_plot = ['-2_-1', '-1_4', '4_1', '1_2', '2_7']
fig, ax = plt.subplots(1, len(pairs_plot), figsize=(3*len(pairs_plot), 2.5))
for a, d1_d2 in zip(ax, pairs_plot):
    sns.stripplot(x='Group', y='cov_z_mean', data=sigz_df[sigz_df['d1_d2'] == d1_d2], ax=a)
    sns.despine(ax=a)
    a.set_title(d1_d2.split('_')[0])
    df_use = sigz_df[sigz_df['d1_d2'] == d1_d2]
    stat, pval = stats.ttest_ind(df_use[df_use['Group'] == 'Ctrl']['cov_z_mean'], 
                                 df_use[df_use['Group'] == 'ANI']['cov_z_mean'])
    
    a.text(0, -0.4, f'p_t = {pval:.3g}')
    
erp.match_max_lims(ax, axis='y')

### Set up figure to contain all covariance plots together, nicely aligned!

In [ ]:
%matplotlib notebook
figcv = plt.figure(figsize=(7, 4))
gs = gridspec.GridSpec(4, 4, figure=figcv)
axfrz_stat = figcv.add_subplot(gs[0,0])
axfrz_plot = figcv.add_subplot(gs[1,0])
axall_stat = figcv.add_subplot(gs[0,2])
axall_plot = figcv.add_subplot(gs[1,2])

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
for ida, (ap, astat, f) in enumerate(zip([axfrz_plot, ax[0]], [axfrz_stat, ax[1]], [figcv, fig])):
    if ida == 0:
        sns.set_context("paper", font_scale=2)
    else:
        sns.set_context("notebook")
    gfg = sns.stripplot(x="Designation", y="cov_z_mean", hue="Group", size=1.25,
                  data=sigz_freeze_df[~(sigz_freeze_df['Designation'] == '')], dodge=True, ax=ap)
    gfg.legend(fontsize=6, markerscale=0.15)
    ap.set_title('Freeze Cell Covariance', fontsize=8)
    ap.set_ylabel(r'$\mathregular{\overline{Z}}$', fontsize=8)
    ap.set_xlabel('', fontsize=8)
    ap.tick_params(axis='both', which='major', labelsize=8, grid_linewidth=1)
    plt.setp(ap.spines.values(), linewidth=1)
    sns.despine(ax=ap)
    stats_before_after(sigz_freeze_df, 'cov_z_mean', ax=astat, fontsize=6)
    sns.despine(ax=astat, bottom=True, left=True)
    astat.tick_params(axis='both', left=False, bottom=False, labelleft=False, labelbottom=False)
fig.savefig(print_dir + "/cov_before_after_freeze_cells.pdf")

Plot #2: Same as #1 but for all cells

In [ ]:
allz_df = cova.cov_dict_to_df(all_cov, baseline_dict_use=all_cov, include_silent=True)
allz_df = add_memory_designations(allz_df)

In [ ]:
allz_df_copy = deepcopy(allz_df)
allz_df_copy.drop(['Designation'], axis=1, inplace=True)
print(allz_df.head(3))
print(allz_df_copy.head(3))
print(add_memory_designations(allz_df_copy).head(3))

In [ ]:
sigz_df = allz_df

pairs_plot = ['-2_-1', '-1_4', '4_1', '1_2', '2_7']
fig, ax = plt.subplots(1, len(pairs_plot), figsize=(3*len(pairs_plot), 2.5))
for a, d1_d2 in zip(ax, pairs_plot):
    sns.stripplot(x='Group', y='cov_z_mean', data=sigz_df[sigz_df['d1_d2'] == d1_d2], ax=a)
    sns.despine(ax=a)
    a.set_title(d1_d2.split('_')[0])
    df_use = sigz_df[sigz_df['d1_d2'] == d1_d2]
    stat, pval = stats.ttest_ind(df_use[df_use['Group'] == 'Ctrl']['cov_z_mean'], 
                                 df_use[df_use['Group'] == 'ANI']['cov_z_mean'])
    
    a.text(0, 0.3, f'p_t = {pval:.3g}')
    
erp.match_max_lims(ax, axis='y')

In [ ]:
from copy import deepcopy
smallfigParams = deepcopy(plt.rcParams)
plotfontsize = 8.0
smallfigParams['axes.labelsize'] = plotfontsize
smallfigParams['axes.linewidth'] = 1.0
smallfigParams['axes.titlesize'] = plotfontsize
smallfigParams['font.size'] = plotfontsize
smallfigParams['legend.fontsize'] = 6
smallfigParams['legend.markerscale'] = 0.25
smallfigParams['xtick.labelsize'] = plotfontsize
smallfigParams['xtick.major.width'] = 1.0
smallfigParams['xtick.major.width'] = 1.0
smallfigParams['ytick.labelsize'] = plotfontsize
smallfigParams['ytick.major.width'] = 1.0
smallfigParams['ytick.major.width'] = 1.0

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib as mpl
fig1, ax1 = plt.subplots(1, 2)

plt.rcParams = smallfigParams
ax1[0].plot([0, 1])
ax1[0].set_xlabel('test small')
plt.rcdefaults

ax1[1].plot([1, 0])
ax1[1].set_xlabel('test normal')

In [ ]:
plt.rcParams

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4))
for ap, astat, f in zip([axall_plot, ax[0]], [axall_stat, ax[1]], [figcv, fig]):
    gfg = sns.stripplot(x="Designation", y="cov_z_mean", hue="Group", size=1.25,
                  data=allz_df[~(allz_df['Designation'] == '')], dodge=True, ax=ap)
    gfg.legend(fontsize=6, markerscale=0.15)
    ap.set_title('All Cell Covariance', fontsize=8)
    ap.set_ylabel(r'$\mathregular{\overline{Z}}$', fontsize=8)
    ap.set_xlabel('', fontsize=8)
    ap.tick_params(axis='both', which='major', labelsize=8, grid_linewidth=1)
    plt.setp(ap.spines.values(), linewidth=1)
    sns.despine(ax=ap)
    stats_before_after(allz_df, "cov_z_mean", astat, fontsize=8)
    sns.despine(ax=astat, bottom=True, left=True)
    astat.tick_params(axis='both', left=False, bottom=False, labelleft=False, labelbottom=False)
fig.savefig(print_dir + "/cov_before_after_all_cells.pdf")

In [ ]:
# NRK - add in ticks and ticklabels manually.
# Next, figure out if you are either setting things manually or using a params file to make plotting easier and
# avoid having to set font sizes separately for each plot!

In [ ]:
# Save to manuscript figure
figcv.savefig(print_dir + "/cov_frz_all_combined_resize.pdf")

Plot #3: Same as #1 and #2 but for all cells on 4 hr session which have covariance > 2std above mean.  This one might be messy due to lots of cell turnover in ANI mice.

In [ ]:
sig4_cov_dict = cova.group_sig_cov_across_days(bin_size=bin_size, arena1='Shock',
                                              arena2='Shock', base_days=[4, 4, 4, 4, 4],
                                              reg_days=[-2, -1, 4, 1, 2], thresh=2,
                                              keep_silent=False)

In [ ]:
# Clean up and reshape dataframe
sig4_dfreg = cova.cov_dict_to_df(sig4_cov_dict, all_cov, register=True)
sig4_dfreg = add_memory_designations(sig4_dfreg, before=['4_-2', '4_-1'], during='4_4', after=['4_1', '4_2'])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4))
sns.swarmplot(x="Designation", y="cov_z_mean", hue="Group", data=sig4_dfreg, dodge=True, ax=ax[0])
ax[0].set_title('4hr sig cells')
sns.despine(fig=fig)
stats_before_after(sig4_dfreg, 'cov_z_mean', ax[1])
fig.savefig(print_dir + "/cov_before_after_sig4_cells_registered_across_days.pdf")

Plot #4: ID freeze cells on 4 hr session and track covariance forward/backward across whole experiment.  Include silent cells in code, but plot two ways: 1) turnover in freeze cell population between sessions, and 2) change in covariance from 4 hr session forward/backward in time.  

In [ ]:
freeze_4hr_reg = cova.group_cov_across_days(bin_size=bin_size, arena1='Shock', arena2='Shock',
                                            neurons='freeze_onset', keep_silent=True,
                                            base_days=[4, 4, 4, 4, 4], reg_days=[-2, -1, 4, 1, 2])

In [ ]:
# Clean up and reshape dataframe
sig4frz_dfreg = cova.cov_dict_to_df(freeze_4hr_reg, all_cov, register=True)
sig4frz_dfreg = add_memory_designations(sig4frz_dfreg, before=['4_-2', '4_-1'], during=['4_4'], after=['4_1', '4_2'])

### Now construct another set of plots tracking freeze cells from 4hr or 1 day session across days

In [ ]:
figfrzreg = plt.figure(figsize=(12, 8))
gs = gridspec.GridSpec(2, 2, figure=figfrzreg)
ax4stat = figfrzreg.add_subplot(gs[0, 0])
ax4plot = figfrzreg.add_subplot(gs[1, 0])
ax1stat = figfrzreg.add_subplot(gs[0, 1])
ax1plot = figfrzreg.add_subplot(gs[1, 1])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4))
for ap, astat, f in zip([ax4plot, ax[0]], [ax4stat, ax[1]], [figfrzreg, fig]):
    sns.swarmplot(x="Designation", y="cov_z_mean", hue="Group", data=sig4frz_dfreg, dodge=True, ax=ap)
    ap.set_title('4 hr freeze cells reg across days')
    ap.set_xlabel('')
    ap.set_ylabel(r'$\overline{Z}$')
    sns.despine(fig=f)
    stats_before_after(sig4frz_dfreg, 'cov_z_mean', astat)
# fig.savefig(print_dir + "/cov_before_after_sig1_cells_registered_across_days.pdf")

Plot #4b: ID freeze cells on 1 day session and track covariance forward/backward across whole experiment.  Include silent cells in code, but plot two ways: 1) turnover in freeze cell population between sessions, and 2) change in covariance from 4 hr session forward/backward in time.  

Plot #5: Same as #4 but do control plot (ID cells on day -1) and plot of consolidated cells (ID cells on day 1).

In [ ]:
sig1_cov_dict = cova.group_sig_cov_across_days(bin_size=bin_size, arena1='Shock',
                                              arena2='Shock', base_days=[1, 1, 1, 1, 1],
                                              reg_days=[-2, -1, 4, 1, 2], thresh=2,
                                              keep_silent=False)

## NRK - look at actual dictionary you created - how many cells? Make sure it isn't just carried by one to two cells that actually register

In [ ]:
# Clean up and reshape dataframe
sig1_dfreg = cova.cov_dict_to_df(sig1_cov_dict, all_cov, register=True)
sig1_dfreg = add_memory_designations(sig1_dfreg, before=['1_-2', '1_-1'], during=['1_4'], after=['1_2'])

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4))
sns.swarmplot(x="Designation", y="cov_z_mean", hue="Group", data=sig1_dfreg, dodge=True, ax=ax[0])
ax[0].set_title('1 day sig cells')
sns.despine(fig=fig)
stats_before_after(sig1_dfreg, 'cov_z_mean', ax[1])
# fig.savefig(print_dir + "/cov_before_after_sig1_cells_registered_across_days.pdf")

Plot #6: More for ANI turnover plots, but plot turnover from day -2 to day -1 for ANI vs Ctrls and compare to day -1 to day 7 - hypothesis is that ANI induces some sort of change in the cells active during later time points, and day -1 to day 7 is the first point at which we see no suppression of activity by ANI.

In [ ]:
sig1frz_cov_dict = cova.group_cov_across_days(bin_size=bin_size, arena1='Shock', neurons='freeze_onset',
                                              arena2='Shock', base_days=[1, 1, 1, 1, 1],
                                              reg_days=[-2, -1, 4, 1, 2], keep_silent=False)
sig2frz_cov_dict = cova.group_cov_across_days(bin_size=bin_size, arena1='Shock', neurons='freeze_onset',
                                              arena2='Shock', base_days=[2, 2, 2, 2, 2],
                                              reg_days=[-2, -1, 4, 1, 2], keep_silent=False)
# Not sure how to interpret keeping silent cells in with across-day registration...
# sig1frz_cov_dict_wsilent = cova.group_cov_across_days(bin_size=bin_size, arena1='Shock', neurons='freeze_onset',
#                                               arena2='Shock', base_days=[1, 1, 1, 1, 1],
#                                               reg_days=[-2, -1, 4, 1, 2], keep_silent=True)

In [ ]:
# Clean up and reshape dataframe
sig1frz_dfreg = cova.cov_dict_to_df(sig1frz_cov_dict, all_cov, register=True)
sig1frz_dfreg = add_memory_designations(sig1frz_dfreg, before=['1_-2', '1_-1'], during=['1_4'], after=['1_1', '1_2'])

sig2frz_dfreg = cova.cov_dict_to_df(sig2frz_cov_dict, all_cov, register=True)
sig2frz_dfreg = add_memory_designations(sig2frz_dfreg, before=['2_-2', '2_-1'], during=['2_4'], after=['2_1', '2_2'])

In [ ]:
sigz_df = sig1frz_dfreg

pairs_plot = ['1_-2','1_-1', '1_4', '1_1', '1_2']
fig, ax = plt.subplots(1, len(pairs_plot), figsize=(3*len(pairs_plot), 2.5))
for a, d1_d2 in zip(ax, pairs_plot):
    sns.stripplot(x='Group', y='cov_z_mean', data=sigz_df[sigz_df['d1_d2'] == d1_d2], ax=a)
    sns.despine(ax=a)
    a.set_title(d1_d2.split('_')[1])
    df_use = sigz_df[sigz_df['d1_d2'] == d1_d2]
    stat, pval = stats.ttest_ind(df_use[df_use['Group'] == 'Ctrl']['cov_z_mean'], 
                                 df_use[df_use['Group'] == 'ANI']['cov_z_mean'])
    
    a.text(0, -0.3, f'p_t = {pval:.3g}')
    
erp.match_max_lims(ax, axis='y')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4))
for ap, astat, f in zip([ax1plot, ax[0]], [ax1stat, ax[1]], [figfrzreg, fig]):
    sns.swarmplot(x="Designation", y="cov_z_mean", hue="Group", data=sig1frz_dfreg, dodge=True, ax=ap)
    ap.set_title('1 day frz cells - Tracked across sessions')
    ap.set_xlabel('')
    ap.set_ylabel(r'$\overline{Z}$')
    sns.despine(fig=f)
    stats_before_after(sig1frz_dfreg, 'cov_z_mean', astat)

In [ ]:
# Save to manuscript figure - supplemental?
figfrzreg.savefig(print_dir + '/cov_frz_1and4_reg.pdf')

In [ ]:
sigz_df = sig2frz_dfreg

pairs_plot = ['2_-2','2_-1', '2_4', '2_1', '2_2']
fig, ax = plt.subplots(1, len(pairs_plot), figsize=(3*len(pairs_plot), 2.5))
for a, d1_d2 in zip(ax, pairs_plot):
    sns.stripplot(x='Group', y='cov_z_mean', data=sigz_df[sigz_df['d1_d2'] == d1_d2], ax=a)
    sns.despine(ax=a)
    a.set_title(d1_d2.split('_')[1])
    df_use = sigz_df[sigz_df['d1_d2'] == d1_d2]
    stat, pval = stats.ttest_ind(df_use[df_use['Group'] == 'Ctrl']['cov_z_mean'], 
                                 df_use[df_use['Group'] == 'ANI']['cov_z_mean'], nan_policy='omit')
    
    a.text(0, -0.3, f'p_t = {pval:.3g}')
    
erp.match_max_lims(ax, axis='y')

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4))
sns.swarmplot(x="Designation", y="cov_z_mean", hue="Group", data=sig2frz_dfreg, dodge=True, ax=ax[0])
ax[0].set_title('2 day frz cells - Tracked across sessions')
sns.despine(fig=fig)
stats_before_after(sig2frz_dfreg, 'cov_z_mean', ax[1])